# Advanced use of the Delaunay-Watershed 3D package.

In this notebook:
- We show how to build custom mesh reconstruction algorithms. You need to understand how the algorithm work to use this.
- We show how to visualize the steps of construction of an algorithm.

# 0) Install Delaunay-Watershed

We install the viewing option from Delaunay-Watershed, based on Napari, to visualize the construction steps later.

In [1]:
pip install "delaunay-watershed-3d[viewing]"


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 1) Custom Mesh Reconstruction Algorithms

Sometimes, we provide some variations in the substeps of the current mesh reconstruction algorithm. These substeps can be selected via a Mesh Reconstruction Algorithm Factory:

In [2]:
from dw3d import MeshReconstructionAlgorithmFactory

# The default algorithm can be obtained with a static method:
default = MeshReconstructionAlgorithmFactory.get_default_algorithm(min_distance=5, print_info=False)


# Other algorithms can be constructed with a factory object:
factory = MeshReconstructionAlgorithmFactory()
# The default algorithm can be recreated from this factory, as it is not modified:
default2 = factory.make_algorithm()

# But you can customize some steps.
# For example, there is a step where we find points on an image, and the min_distance argument
# is important for this step. It can be customized:
factory.set_peak_local_points_placement_method(min_distance=3)
custom_algorithm = factory.make_algorithm()
# Now the custom algorithm makes meshes with more points than the default algorithm.

# For the next algorithm, we change the method for score computation:
factory.set_score_computation_by_max_value()
custom_algorithm2 = factory.make_algorithm()


# An algorithm can be created very quickly by chaining operations:
custom_algorithm3 = (
    MeshReconstructionAlgorithmFactory(print_info=True, perform_mesh_postprocess_surgery=True)
    .set_classical_edt_method()
    .set_peak_local_points_placement_method(min_distance=4)
    .set_delaunay_tesselation_method()
    .set_score_computation_by_mean_value()
    .make_algorithm()
)

 All the algorithm use the same API than the default algorithm. For example:

In [3]:
import skimage.io as io
image_path = "../data/Images/1.tif"
image = io.imread(image_path)
points, triangles, labels =  custom_algorithm3.construct_mesh_from_segmentation_mask(image)

Computing EDT ...
EDT computed in  0.51
Searching local extremas ...
Number of local minimas : 2940
Number of local maxes : 84
Delaunay Tesselation built in 0.11 seconds
Graph build in  0.322
Watershed done in 0.23 seconds.


# 2) Visualize the construction process with Napari

As we give the algorithm to the viewing function, we have access to the steps of the construction process and we can visualize them:

In [4]:
from dw3d.viewing import plot_in_napari

# Be sure you computed a mesh first with the algorithm of course !
# The last mest computed is shown.
v = plot_in_napari(custom_algorithm3, add_mesh=True, original_image=None)  # you can also see original image (not the mask) here

/home/perez/Documents/Sacha/delaunay_watershed_3D/.venv/lib/python3.11/site-packages/napari/layers/utils/interaction_box.py:53: RuntimeWarning: overflow encountered in scalar subtract
  box_height = vertices[0, 1] - vertices[1, 1]
